In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torchvision 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import pandas as pd
import pdb
import os
import shutil
import pyrealsense2 as rs


In [6]:
finetuned_fan_pixel_path = r'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/LANDMARKS/Diego_FaceData/landmarks_finetuned_w15/AF18_02_20190826_NSM_OPEN_landmarksFiltered.csv'
df_fan = pd.read_csv(finetuned_fan_pixel_path, index_col=0)

finetuned_fan_path = r'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/LANDMARKS/Diego_FaceData/landmarks_finetuned_w15/AF18_02_20190826_NSM_OPEN_landmarksFiltered3D.csv'
df_fan3D = pd.read_csv(finetuned_fan_path, index_col=0)


mobileNet_pixel_path = r'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/LANDMARKS/mobileNet/AF18_02_20190826_NSM_OPEN_color_landmarksPixels2D.csv'
df_mnet = pd.read_csv(mobileNet_pixel_path, index_col=0)


mobileNet_path = r'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/LANDMARKS/mobileNet/AF18_02_20190826_NSM_OPEN_color_landmarks3D.csv'
df_mnet3D = pd.read_csv(mobileNet_path, index_col=0)


In [7]:
df_fan.head()

,Video_Frame_number,Time_Stamp (s),bbox_top_x,bbox_top_y,bbox_bottom_x,bbox_bottom_y,landmark_0_x,landmark_0_y,landmark_1_x,landmark_1_y,...,landmark_63_x,landmark_63_y,landmark_64_x,landmark_64_y,landmark_65_x,landmark_65_y,landmark_66_x,landmark_66_y,landmark_67_x,landmark_67_y
0,0,0.000000,248.503937,121.724037,424.76712,356.467682,233.0,214.0,237.0,235.0,...,357.0,281.0,367.0,281.0,357.0,280.0,348.0,281.0,339.0,281.0
1,1,0.033399,248.503937,121.724037,424.76712,356.467682,236.0,215.0,242.0,235.0,...,357.0,282.0,367.0,281.0,357.0,280.0,348.0,282.0,339.0,282.0
2,2,0.066799,248.503937,121.724037,424.76712,356.467682,237.0,215.0,242.0,236.0,...,357.0,285.0,367.0,285.0,358.0,282.0,348.0,283.0,339.0,285.0
3,3,0.100198,248.503937,121.724037,424.76712,356.467682,249.0,222.0,253.0,246.0,...,359.0,286.0,367.0,289.0,359.0,289.0,349.0,291.0,339.0,292.0
4,4,0.133597,248.503937,121.724037,424.76712,356.467682,249.0,222.0,253.0,246.0,...,359.0,286.0,368.0,289.0,359.0,289.0,349.0,291.0,340.0,292.0


In [8]:
df_fan3D.head()

,Video_Frame_number,Time_Stamp (s),landmark_0,landmark_0.1,landmark_0.2,landmark_1,landmark_1.1,landmark_1.2,landmark_2,landmark_2.1,...,landmark_64.2,landmark_65,landmark_65.1,landmark_65.2,landmark_66,landmark_66.1,landmark_66.2,landmark_67,landmark_67.1,landmark_67.2
NaN,NaN,NaN,x,y,z,x,y,z,x,y,...,z,x,y,z,x,y,z,x,y,z
0.0,0.0,0.000000,-0.07900119184788792,-0.022928319835558264,0.636806967886514,-0.07465074095975358,-0.0013958296456191642,0.634682194882771,-0.05827559817463226,0.01695013900531364,...,0.5101954941928852,0.03819961123493017,0.03551226923941896,0.5055709882435622,0.03072975680806653,0.03615550966259582,0.5031962419452611,0.023421865981740588,0.03612856815017958,0.5028212820034241
1.0,1.0,0.033399,-0.07679010620692442,-0.022152525183323755,0.6440561934286961,-0.06217437198776595,-0.0012479453704432857,0.567439378646668,-0.055595814123016236,0.01696513581044995,...,0.5109454140765592,0.03816183658846794,0.035477152038687766,0.5050710416544462,0.030706858330564246,0.036938011664961745,0.5028212820034241,0.023421865981740588,0.036938011664961745,0.5028212820034241
2.0,2.0,0.066799,-0.07585620782834353,-0.022182617882000887,0.6449310999596491,-0.062475657490790364,-0.0003361002357724784,0.5701890848868061,-0.05620809676996262,0.022116533167833897,...,0.5104454674874432,0.0390134813167097,0.03714000924304506,0.5055709882435622,0.03076028811140291,0.03781313556409843,0.5036961885343771,0.023503373842477443,0.039503336688783856,0.5045710950653302
3.0,3.0,0.100198,-0.05638214165978376,-0.013264638784393122,0.5735637243633391,-0.051322536810267806,0.008664494592003986,0.5586903133371379,-0.05189961348332196,0.029961496600307492,...,0.5118203206075123,0.04027042452900396,0.04331365536612158,0.5111953873711172,0.031821829087460694,0.04465171250223247,0.5076957612473052,0.023602347673372197,0.04537945299006317,0.5066958680690732


In [9]:
df_mnet.head()

,landmark_0_x,landmark_0_y,landmark_1_x,landmark_1_y,landmark_2_x,landmark_2_y,landmark_3_x,landmark_3_y,landmark_4_x,landmark_4_y,...,landmark_63_y,landmark_64_x,landmark_64_y,landmark_65_x,landmark_65_y,landmark_66_x,landmark_66_y,landmark_67_x,landmark_67_y,fps
0,252.533020,228.426147,255.125427,249.331360,260.613464,273.179016,267.364685,293.252441,277.426178,313.968903,...,283.967743,370.816498,289.341949,364.352814,290.780151,352.017517,292.938751,338.157990,293.367065,29.0
1,252.494827,227.878662,254.939499,249.512024,260.217590,273.447693,267.161957,294.119812,277.399139,314.369202,...,283.814209,369.992096,288.890686,363.508209,290.606964,351.352844,293.196838,338.066895,293.707825,29.0
2,252.825714,231.931946,255.348953,253.767059,260.903931,277.970337,267.786896,297.595642,277.748199,317.597595,...,284.899414,371.223389,290.498962,364.760376,293.146942,353.049988,294.636719,339.486023,294.871887,29.0
3,254.988907,230.836273,258.376099,252.938446,264.360382,275.079895,271.816223,295.790802,282.341949,316.810120,...,283.825928,370.055542,288.622772,364.850952,292.399628,353.019470,294.168274,339.828979,294.952393,29.0
4,254.952118,229.641235,258.135925,252.797882,263.766357,275.155090,271.664001,296.077393,282.223389,317.380249,...,284.117157,371.014801,289.104889,365.085266,292.332245,352.757874,294.297455,338.733643,295.655762,29.0


In [10]:
df_mnet3D.head()

,Time_Stamp (s),landmark_0,landmark_0.1,landmark_0.2,landmark_1,landmark_1.1,landmark_1.2,landmark_2,landmark_2.1,landmark_2.2,...,landmark_65,landmark_65.1,landmark_65.2,landmark_66,landmark_66.1,landmark_66.2,landmark_67,landmark_67.1,landmark_67.2,depth_scale
NaN,NaN,x,y,z,x,y,z,x,y,z,...,x,y,z,x,y,z,x,y,z,NaN
0.0,0.000000,-0.051603623971284755,-0.007121935727317386,0.5571904735697899,-0.04902407952791856,0.011569355936594514,0.5543157806823729,-0.04830575738759869,0.035960456976643435,0.6068101725395536,...,0.04469723095664575,0.04480048158850613,0.5114453606656753,0.034389311282483674,0.04637282693416097,0.5091956010146532,0.022926840926114987,0.04651747967780729,0.5069458413636312,0.000125
1.0,0.033399,-0.05198537745778456,-0.007664243725355044,0.56094007298816,-0.04965582607139687,0.01184165957294459,0.5595652198680909,-0.04876267047554594,0.03627513922998659,0.6076850790705066,...,0.04403410497133751,0.044690632306066735,0.5118203206075123,0.03363678983901755,0.04629851833986847,0.5060709348326782,0.022846865593963567,0.046784030245996464,0.5068208547163522,0.000125
2.0,0.066799,-0.05160596916916313,-0.0039978696787543455,0.5600651664572069,-0.04944111867350938,0.015723569185050544,0.561315032929997,-0.0486352742129414,0.04115983301624642,0.6145593446708517,...,0.0449667573503345,0.04668057625659476,0.5106954407820012,0.03501941009078494,0.04747155261860814,0.5060709348326782,0.023975108199337427,0.047674910021175285,0.5061959214799572,0.000125
3.0,0.100198,-0.04873589117074952,-0.00489337374321737,0.5496912747330498,-0.04590497956968739,0.01471807899944084,0.5516910610895138,-0.04069140534877608,0.03446789851692135,0.5530659142095828,...,0.04511838491327401,0.04614511365395639,0.5115703473129543,0.03503776200193505,0.047148072481592966,0.5066958680690732,0.02425457529010995,0.047740511917689014,0.5061959214799572,0.000125
